Импорт необходимых библиотек

In [1]:
import cupy as cp
import numpy as np
import time

Ядро вычислений, на С++

In [2]:
add_kernel = cp.RawKernel(r'''
extern "C" __global__
void matmul(const float* a, const float* b, float* c, int width) {
    int x = blockDim.x * blockIdx.x + threadIdx.x;
    int y = blockDim.y * blockIdx.y + threadIdx.y;

    float value = 0.0, elem1 = 0.0, elem2 = 0.0;
    
    for(int i = 0; i < width; i++)
	{
		elem1 = a[y * width + i];
		elem2 = b[i * width + x];
		
		value += elem1 * elem2;
	}
                      
    c[y * width + x] = value;
}
''',
"matmul")

Задаваемые значения

In [3]:
p = 2000
q = 2000
N = 4000000

Матрицы для вычисления на GPU, значения 0 -> N-1

In [4]:
a = cp.arange(N, dtype=cp.float32).reshape(p,q)
b = cp.arange(N, dtype=cp.float32).reshape(p,q)
res = cp.zeros((p,q), dtype=cp.float32)
width = p

Матрицы для вычисления на GPU, значения 1

In [5]:
a = cp.ones((p,q), dtype=cp.float32)
b = cp.ones((p,q), dtype=cp.float32)
res = cp.zeros((p,q), dtype=cp.float32)
width = p

Вычисления на GPU, подсчёт скорости вычислений

In [6]:
gstart = time.perf_counter()
result = add_kernel((200, 200), (10, 10), (a, b, res, width))
gend = time.perf_counter()

print(gend - gstart)

0.006152200046926737


Выполнение матричного умножения на CPU, подсчёт скорости вычислений

In [7]:
def cpu_matmul(a, b, n):
    c = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            c[i, j] = 0
            for k in range(n):
                c[i, j] += a[i, j]*b[i, j]
    return c


x = np.arange(1000000, dtype=np.float32).reshape(1000,1000)
y = np.arange(1000000, dtype=np.float32).reshape(1000,1000)
z = 1000

cstart = time.perf_counter()
cpu_matmul(x, y, z)
cend = time.perf_counter()

print(cend-cstart)

KeyboardInterrupt: 